# Ranking and Position Sizing

In this notebook, we will learn about the features of **PyBroker** that enable you to rank ticker symbols and set position sizes for a group of symbols in your trading strategy. With these features, you can easily optimize your strategy and manage risk more effectively.

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('ranking_and_pos_sizing')

## Ranking Ticker Symbols

In this section, we will learn about how to rank ticker symbols when placing buy orders. Let's begin with an example of how to rank ticker symbols based on volume when placing buy orders. 

In [2]:
def buy_highest_volume(ctx):
    # If there are no long positions across all tickers being traded:
    if not tuple(ctx.long_positions()):
        ctx.buy_shares = ctx.calc_target_shares(1)
        ctx.hold_bars = 2
        ctx.score = ctx.volume[-1]

The ```buy_highest_volume``` function ranks ticker symbols by their most recent trading volume and allocates 100% of the portfolio for 2 bars. The ```ctx.score``` is set to ```ctx.volume[-1]```, which is the most recent trading volume.

In [3]:
config = StrategyConfig(max_long_positions=1)
strategy = Strategy(YFinance(), '6/1/2021', '6/1/2022', config)
strategy.add_execution(buy_highest_volume, ['T', 'F', 'GM', 'PFE'])

To limit the number of long positions that can be held at any time to ```1```, we set [max_long_positions](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.max_long_positions) to ```1``` in the [StrategyConfig](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig). In this example, we add the ```buy_highest_volume``` function to the [Strategy](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) object and specify the ticker symbols to trade: ```['T', 'F', 'GM', 'PFE']```.

In [4]:
result = strategy.backtest()
result.trades

Backtesting: 2021-06-01 00:00:00 to 2022-06-01 00:00:00

Loading bar data...
[*********************100%***********************]  4 of 4 completed
Loaded bar data: 0:00:01 

Test split: 2021-06-01 00:00:00 to 2022-05-31 00:00:00


100% (253 of 253) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:02


,type,symbol,entry_date,exit_date,entry,exit,shares,pnl,return_pct,agg_pnl,bars,pnl_per_bar,stop
id,,,,,,,,,,,,,
1,long,F,2021-06-02,2021-06-04,14.85,16.13,6734,8619.52,8.62,8619.52,2,4309.76,bar
2,long,F,2021-06-07,2021-06-09,15.93,15.51,6801,-2856.42,-2.64,5763.10,2,-1428.21,bar
3,long,F,2021-06-10,2021-06-14,15.43,15.06,6832,-2527.84,-2.40,3235.26,2,-1263.92,bar
4,long,F,2021-06-15,2021-06-17,14.96,14.99,6900,207.00,0.20,3442.26,2,103.50,bar
5,long,F,2021-06-18,2021-06-22,14.61,14.96,7003,2451.05,2.40,5893.31,2,1225.53,bar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,long,F,2022-05-10,2022-05-12,13.43,12.47,7263,-6972.48,-7.15,-9423.13,2,-3486.24,bar
81,long,F,2022-05-13,2022-05-17,13.25,13.34,6835,615.15,0.68,-8807.98,2,307.58,bar
82,long,F,2022-05-18,2022-05-20,13.03,12.59,6739,-2965.16,-3.38,-11773.14,2,-1482.58,bar


## Setting Position Sizes

In **PyBroker**, you can set position sizes based on multiple tickers. To illustrate this, let's take a simple buy and hold strategy that starts trading after 100 days and holds positions for 30 days:

In [5]:
def buy_and_hold(ctx):
    if not ctx.long_pos() and ctx.bars >= 100:
        ctx.buy_shares = 100
        ctx.hold_bars = 30
        
strategy = Strategy(YFinance(), '6/1/2021', '6/1/2022')
strategy.add_execution(buy_and_hold, ['T', 'F', 'GM', 'PFE'])

This will buy ```100``` shares in each of ```['T', 'F', 'GM', 'PFE']```. But what if you don't want to use equal position sizing? For example, you may want to size positions so that more shares are allocated to tickers with lower volatility to decrease the portfolio's overall volatility.

To customize position sizing for each ticker, we can define a [pos_size_handler](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.set_pos_size_handler) function that calculates the position size for each ticker:

In [6]:
import numpy as np

def pos_size_handler(ctx):
    # Fetch all buy signals.
    signals = tuple(ctx.signals("buy"))
    # Return if there are no buy signals (i.e. there are only sell signals).
    if not signals:
        return
    # Calculates the inverse volatility, where volatility is defined as the
    # standard deviation of close prices for the last 100 days.
    get_inverse_volatility = lambda signal: 1 / np.std(signal.bar_data.close[-100:])
    # Sums the inverse volatilities for all of the buy signals.
    total_inverse_volatility = sum(map(get_inverse_volatility, signals))
    for signal in signals:
        size = get_inverse_volatility(signal) / total_inverse_volatility
        # Calculate the number of shares given the latest close price.
        shares = ctx.calc_target_shares(size, signal.bar_data.close[-1])
        ctx.set_shares(signal, shares)
        
strategy.set_pos_size_handler(pos_size_handler)

The handler runs on every bar that generates a buy or sell signal when [buy_shares](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.buy_shares) or [sell_shares](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.sell_shares) is set on the [ExecContext](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext):

In [7]:
result = strategy.backtest()

Backtesting: 2021-06-01 00:00:00 to 2022-06-01 00:00:00

Loaded cached bar data.

Test split: 2021-06-01 00:00:00 to 2022-05-31 00:00:00


100% (253 of 253) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


In [8]:
result.trades

,type,symbol,entry_date,exit_date,entry,exit,shares,pnl,return_pct,agg_pnl,bars,pnl_per_bar,stop
id,,,,,,,,,,,,,
1,long,F,2021-10-21,2021-12-03,16.41,19.66,2135,6938.75,19.80,6938.75,30,231.29,bar
2,long,PFE,2021-10-21,2021-12-03,42.76,53.75,248,2725.52,25.70,9664.27,30,90.85,bar
3,long,T,2021-10-21,2021-12-03,19.60,17.54,2366,-4873.96,-10.51,4790.31,30,-162.47,bar
4,long,GM,2021-10-21,2021-12-03,58.20,60.28,136,282.88,3.57,5073.19,30,9.43,bar
5,long,F,2021-12-06,2022-01-19,19.05,23.66,1189,5481.29,24.20,10554.48,30,182.71,bar
6,long,PFE,2021-12-06,2022-01-19,52.57,53.97,320,448.00,2.66,11002.48,30,14.93,bar
7,long,T,2021-12-06,2022-01-19,17.81,20.49,2948,7900.64,15.05,18903.12,30,263.35,bar
8,long,GM,2021-12-06,2022-01-19,59.72,57.99,219,-378.87,-2.90,18524.25,30,-12.63,bar
9,long,F,2022-01-20,2022-03-04,22.22,17.01,985,-5131.85,-23.45,13392.40,30,-171.06,bar


Using this method allows for a lot of possibilities, such as using [Mean-Variance Optimization](https://en.wikipedia.org/wiki/Modern_portfolio_theory) to determine portfolio allocations. 

[In the next notebook, we will discuss how to implement custom indicators in PyBroker](https://www.pybroker.com/en/latest/notebooks/5.%20Writing%20Indicators.html).